In [10]:
import math, sys
from konlpy.tag import Okt

class BayesianFilter:
  """ 베이지안 필터 """
  def __init__(self):
    self.words = set() # 출현 단어 기록 
    self.word_dict = {} # 카테고리 마다 출현 횟수 기록
    self.category_dict = {} # 카테고리 출현 횟수 기록 

    # 형태소 분석하기 
  def split(self, text): 
    results = []
    okt = Okt() 
    # 단어의 기본형 사용
    malist = okt.pos(text, norm=True, stem=True) 
    for word in malist: 
        # 어미/조사/구두점 등을 대상에서 제외  
      if not word[1] in ["Josa","Eomi","Punctuation"]:
        results.append(word[0]) 
    return results 

    # 단어와 카테고리의 출현 횟수 세기
  def inc_word(self, word, category):
    # 단어를 카테고리에 추가하기
    if not category in self.word_dict: 
      self.word_dict[category] = {} 
    if not word in self.word_dict[category]: 
      self.word_dict[category][word] = 0
      self.word_dict[category][word] += 1 
      self.words.add(word) 

  def inc_category(self, category):
    # 카테고리 계산
    if not category in self.category_dict: 
      self.category_dict[category] = 0
      self.category_dict[category] += 1 

  # 텍스트 학습하기
  def fit(self, text, category):
    """ 텍스트 학습 """ 
    word_list = self.split(text) 
    for word in word_list:
      self.inc_word(word, category) 
    self.inc_category(category)  
  
  # 단어 리스트에 점수 매기기 
  def score(self, words, category):
    score = math.log(self.category_prob(category)) 
    for word in words:
      score += math.log(self.word_prob(word, category)) 
    return score

  # 카테고리 내부의 단어 출현 횟수 구하기
  def get_word_count(self, word, category): 
    if word in self.word_dict[category]:
      return self.word_dict[category][word]
    else:
      return 0

  def category_prob(self, category): 
    """ 카테고리 계산 """ 
    sum_categories = sum(self.category_dict.values()) 
    category_v = self.category_dict[category] 
    return category_v / sum_categories 

  def word_prob(self, word, category):
    """ 카테고리 내부의 단어 출현 비율을 계산 """
    n = self.get_word_count(word, category) + 1 # 0은 의미가 없으므로 +1을 해야한다.
    d = sum(self.word_dict[category].values()) + len(self.words) 
    return n / d 

  def predict(self, text): 
    best_category = None
    max_score = -sys.maxsize 
    words = self.split(text) 
    score_list = [] 
    for category in self.category_dict.keys():
      score = self.score(words, category)
      score_list.append((category, score))
      if score > max_score:
        max_score = score
        best_category = category 
    return best_category, score_list 


In [ ]:
# from bayes import BayesianFilter => 모듈을 호출하여 BayesianFilter를 로드... 

In [11]:
bf = BayesianFilter() # 클래스 호출(객체 생성) 

## 텍스트 학습
bf.fit("파격 세일 - 오늘까지만 30%할인", "광고") 
bf.fit("봄과 함께 찾아온 따뜻한 신제품 소식", "광고") 
bf.fit("인기 제품 기간 한정 세일", "광고") 
bf.fit("쿠폰 선물 & 무료 배송", "광고") 
bf.fit("신세계 백화점 세일", "광고")
bf.fit("오늘 일정 확인", "중요")
bf.fit("프로젝트 진행 상황 보고", "중요")
bf.fit("계약 잘 부탁드립니다", "중요")
bf.fit("회의 일정이 등록되었습니다.", "중요") 
bf.fit("오늘 일정이 없습니다.", "중요")

In [13]:
# 예측 
pre, score_list = bf.predict("계약은 오늘 회의에서 결정됩니다") 
print("결과 예측 =", pre) 
print(score_list)  

결과 예측 = 중요
[('광고', -20.21525633917275), ('중요', -18.072807129433244)]


In [ ]:
import os, glob, json
root_dir = "./newstext"
dic_file = root_dir + "/word-dic.json"
data_file = root_dir + "/data.json" 
data_file_min = root_dir + "/data-mini.json" 

## 어구를 자르고 ID로 변환
word_dic = {"_MAX": 0} 
def text_to_ids(text): 
  text = text.strip()
  words = text.split(" ")
  result = []
  for n in words:
    n = n.strip()
    if n == "" : continue
    if not n in word_dic:
      wid = word_dic[n] = word_dic["_MAX"] 
      word_dic["_MAX"] += 1
      print(wid, n)
    else:
      wid = word_dic[n]
    result.append(wid) 
  return result # 단어의 사용 횟수를 알수 있다. 

# 파일을 읽고 고정 길이의 배열 리턴하기  
def file_to_ids(fname):
  with open(fname, "r", encoding="utf-8") as f: 
    text = f.read()
  return text_to_ids(text)

# 딕셔너리에 단어 모두 등록하기
def register_dic():
  files = glob.glob(root_dir+"/*/*.wakati", recursive=True) 
  for i in files:
    file_to_ids(i)

# 파일 내부의 단어 세기
def count_file_freq(fname):
  cnt = [0 for n in range(word_dic["_MAX"])] 
  with open(fname,"r", encoding="utf-8") as f:
    text = f.read().strip() 
    ids = text_to_ids(text) 
    for wid in ids:
      cnt[wid] += 1 
  return cnt

# 카테고리마다 파일 읽어 들이기 
def count_freq(limit = 0): 
  X = []
  Y = []
  max_words = word_dic["_MAX"]
  cat_names = []
  for cat in os.listdir(root_dir):
    cat_dir = root_dir + "/" + cat
    if not os.path.isdir(cat_dir): continue # 내용이 없다면
    cat_idx = len(cat_names)  
    cat_names.append(cat)
    files = glob.glob(cat_dir+"/*.wakati") 
    i = 0
    for path in files: 
      print(path)
      cnt = count_file_freq(path) 
      X.append(cnt)
      Y.append(cat_idx) 
      if limit > 0:
        if i > limit: break
        i += 1 
  return X,Y

# 단어 딕셔너리 만들기 
if os.path.exists(dic_file):
  word_dic =json.load(open(dic_file)) 
else:
  register_dic()
  json.dump(word_dic, open(dic_file,"w",encoding="utf-8"))

# 벡터를파일로출력하기 
# 테스트 목적의 소규모 데이터 만들기
X, Y = count_freq(20)
json.dump({"X": X, "Y": Y}, open(data_file_min,"w",encoding="utf-8"))
# 전체 데이터를 기반으로 데이터 만들기
X, Y = count_freq()
json.dump({"X": X, "Y": Y}, open(data_file,"w",encoding="utf-8")) 
print("ok")



In [ ]:
### keras를 이용한 MLP 텍스트 분류하기
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation 
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import json   

max_words = 77238
nb_classes = 6 # 9개의 카테고리

batch_size = 64 
nb_epoch = 20

# MLP 모델 생성하기 
def build_model():
  model = Sequential()
  model.add(Dense(512, input_shape=(max_words,))) 
  model.add(Activation('relu'))
  model.add(Dropout(0.5)) 
  model.add(Dense(nb_classes))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy']) 
  return model

# 데이터 읽어 들이기 
data = json.load(open("./newstext/data-mini.json")) 
#data = json.load(open("./newstext/data.json"))
X = data["X"] 
Y = data["Y"] 


In [ ]:
# 학습하기 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y) 
Y_train = np_utils.to_categorical(Y_train, nb_classes) 
print(len(X_train),len(Y_train))
model = KerasClassifier( 
  build_fn=build_model, 
  nb_epoch=nb_epoch, 
  batch_size=batch_size)
model.fit(X_train, Y_train)

#예측하기 
y = model.predict(X_test)
ac_score = metrics.accuracy_score(Y_test, y) 
cl_report = metrics.classification_report(Y_test, y) 
print("정답률 =", ac_score)
print("리포트 =\n", cl_report)

In [7]:
### 레벤슈타인 거리 구하는 프로그램 
def calc_distance(a, b):
  """ 레벤슈타인 거리 계산하기 """
  if a == b: return 0
  a_len = len(a) 
  b_len = len(b)
  if a == "": return b_len 
  if b == "": return a_len

#2차원표(a_len+1,b_len+1)
  matrix = [[] for i in range(a_len+1)]
  for i in range(a_len+1): # 0으로 초기화
    matrix[i] = [0 for j in range(b_len+1)] # 0일 때 초깃값을 설정

  for i in range(a_len+1):
    matrix[i][0] = i
  for j in range(b_len+1): matrix[0][j] = j

#표 채우기 
  for i in range(1, a_len+1):
    ac = a[i-1]
    for j in range(1, b_len+1):
      bc = b[j-1]
      cost = 0 if (ac == bc) else 1
      matrix[i][j] = min ([
        matrix[i-1][j] + 1, # 문자 삽입 
        matrix[i][j-1] + 1, # 문자 제거 
        matrix[i-1][j-1] + cost # 문자 변경

      ])
  return matrix[a_len][b_len]
# "가나다라"와 "가마바라"의 거리
print(calc_distance("가나다라", "가마바라"))
# 실행 예
samples = ["신촌역","신천군","신천역","신발","마곡역"]
base = samples[0]
r = sorted(samples, key = lambda n: calc_distance(base, n)) 
for n in r:
  print(calc_distance(base, n), n)


 

2
0 신촌역
1 신천역
2 신천군
2 신발
2 마곡역
